In [1]:
! gdown https://drive.google.com/u/3/uc?id=1--qv7im7ND4OMJamMZy2WY43Kw4-7SjX

Downloading...
From: https://drive.google.com/u/3/uc?id=1--qv7im7ND4OMJamMZy2WY43Kw4-7SjX
To: /content/znz-classify.zip
495MB [00:09, 53.7MB/s]


In [2]:
! unzip -qq znz-classify.zip

In [19]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model
from keras import backend as K 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import glob


#! unzip -qq roads.zip
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.models import Model , load_model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision 
from tensorflow.keras import backend as K


In [20]:
import glob
all_img_files = glob.glob('znz-segment-z19/znz-train-z19-all-buffered/images-512/*')
all_mask_files = glob.glob('znz-segment-z19/znz-train-z19-all-buffered/masks-512/*')
print(len(all_img_files))
print(len(all_mask_files))
print(all_img_files[:10])
print(all_mask_files[:10])

#img = cv2.imread('trainx/X_img_144.bmp', cv2.IMREAD_COLOR)
#img.shape

#plt.imshow(img[:,:,::-1])

img_files = glob.glob('znz-segment-z19/znz-train-z19-all-buffered/images-512/*')
msk_files = glob.glob('znz-segment-z19/znz-train-z19-all-buffered/masks-512/*')

img_files.sort()
msk_files.sort()

print(len(img_files))
print(len(msk_files))


2691
2691
['znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_043_corrected_19_319464_270792_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_043_corrected_19_319464_270800_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_023_19_319450_270672_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_050_19_319476_270834_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_043_corrected_19_319458_270786_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_043_corrected_19_319467_270798_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_050_19_319470_270811_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_029_corrected_19_319457_270681_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_029_corrected_19_319456_270704_img.jpg', 'znz-segment-z19/znz-train-z19-all-buffered/images-512/grid_022_19_319424_270664_img.jpg']
['znz-segment-z19/z

In [21]:
X_name = []
Y_name = []

for img_fl in tqdm(img_files):
  #print(img_fl)
  #break
  #img = cv2.imread('{}'.format(img_fl), cv2.IMREAD_COLOR)
  #resized_img = cv2.resize(img,(256, 256), interpolation = cv2.INTER_CUBIC)

  #X.append(resized_img)
  X_name.append(img_fl)
  mask_name = 'znz-segment-z19/znz-train-z19-all-buffered/masks-512/'+str(img_fl.split('.')[0][:-4]).split('/')[-1]+"_mask_buffered.png"
  #print("mn = ",mask_name)
  #break
  #msk = cv2.imread('{}'.format(mask_name), cv2.IMREAD_GRAYSCALE)
  #resized_msk = cv2.resize(msk,(256, 256), interpolation = cv2.INTER_CUBIC)
  Y_name.append(mask_name)
  #Y.append(resized_msk)

print(len(X_name))
print(len(Y_name))

100%|██████████| 2691/2691 [00:00<00:00, 282404.79it/s]

2691
2691


In [22]:
X_train_name, X_test_name, Y_train_name, Y_test_name = train_test_split(X_name, Y_name, test_size=0.2, random_state=3)

In [23]:
def read_img(path):
    xt = cv2.imread(path, cv2.IMREAD_COLOR)
    xt = cv2.cvtColor(xt, cv2.COLOR_BGR2RGB)
    xt = cv2.resize(xt, (IMG_SIZE, IMG_SIZE))
    xt = xt/255.0
    return xt

def read_mask(path):
    xt = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    xt = cv2.resize(xt, (IMG_SIZE, IMG_SIZE))
    xt = np.expand_dims(xt, axis=-1)
    xt = xt/255.0
    return xt

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [10]:
! mkdir results

In [24]:
IMG_SIZE = 256

In [25]:
def model():
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="input_image")
    
    encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=0.50)
    skip_connection_list = ["input_image", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu"]
    encoder_output = encoder.get_layer("block_13_expand_relu").output
    
    fil = [48, 64, 96, 128]
    xt = encoder_output
    for i in range(1, len(skip_connection_list)+1, 1):
        xt_skip = encoder.get_layer(skip_connection_list[-i]).output
        xt = UpSampling2D((2, 2))(xt)
        xt = Concatenate()([xt, xt_skip])
        
        xt = Conv2D(fil[-i], (3, 3), padding="same")(xt)
        xt = BatchNormalization()(xt)
        xt = Activation("relu")(xt)
        
        xt = Conv2D(fil[-i], (3, 3), padding="same")(xt)
        xt = BatchNormalization()(xt)
        xt = Activation("relu")(xt)
        
    xt = Conv2D(1, (1, 1), padding="same")(xt)
    xt = Activation("sigmoid")(xt)
    
    model = Model(inputs, xt)
    return model

In [26]:
model = model()
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 257, 257, 3)  0           input_image[0][0]                
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 16) 432         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 16) 64          Conv1[0][0]                      
_______________________________________________________________________________________

In [27]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [29]:
# Run this module only while loading the pre-trained model.
model = load_model('Unet_with_weight_150e.h5',custom_objects={'dice_loss': dice_loss,'dice_coef':dice_coef})
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 257, 257, 3)  0           input_image[0][0]                
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 16) 432         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 16) 64          Conv1[0][0]                      
____________________________________________________________________________________________

In [32]:
for i, (a, b) in enumerate(zip(X_test_name[0:10], Y_test_name[0:10])):
    a = read_img(a)
    b = read_mask(b)
    y_pred = model.predict(np.expand_dims(a, axis=0))[0] > 0.5
    h, w, _ = a.shape
    white_line = np.ones((h, 10, 3))

    all_images = [
        a, white_line,
        mask_parse(b), white_line,
        mask_parse(y_pred)
    ]
    image = np.concatenate(all_images, axis=1)
    
    fig = plt.figure(figsize=(12, 12))
    x = fig.add_subplot(1, 1, 1)
    imgplot = plt.imshow(image)
    plt.savefig('results/'+str(i)+'.png',format='png')
    plt.close()

In [33]:
! zip -r results.zip results

  adding: results/ (stored 0%)
  adding: results/4.png (deflated 4%)
  adding: results/3.png (deflated 4%)
  adding: results/7.png (deflated 6%)
  adding: results/1.png (deflated 5%)
  adding: results/9.png (deflated 5%)
  adding: results/6.png (deflated 5%)
  adding: results/2.png (deflated 4%)
  adding: results/5.png (deflated 4%)
  adding: results/0.png (deflated 5%)
  adding: results/8.png (deflated 5%)
